In [15]:
import numpy as np
from collections import Counter
import pandas as pd


#conver X_train and X_test to numerical values instead of string
def convert(df):
    buying_dict = {'low':1, 'med': 2, 'high':3, 'vhigh':4}
    lug_dict = {'small':1, 'med':2, 'big':3}
    safety_dict = {'low': 1, 'med':2, 'high':3}
    doors_dict = {'1':1, '2':2, '3':3, '4':4, '5more': 5}
    persons_dict = { '2':2, '4':4, 'more': 6}
    df['buying'] = df['buying'].map(buying_dict)
    df['maint'] = df['maint'].map(buying_dict)
    df['lug_boot'] = df['lug_boot'].map(lug_dict)
    df['safety'] = df['safety'].map(safety_dict)
    df['doors'] = df['doors'].map(doors_dict)
    df['persons'] = df['persons'].map(persons_dict)
    return(df)

def entropy(y):
    #y[1,1,2,3]
    #counts the occurrence of each element
    hist = np.bincount(y)
    #[1,2...8]
    ps = hist / len(y)
    #apply formula for entropy
    ent =-np.sum([p * np.log2(p) for p in ps if p > 0])
    return ent

#class for Node
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

#leaf nodes are nodes right at the end (when decisions are made)
    def is_leaf(self):
        return self.value is not None


In [16]:
#class for Decision Trees
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=100, Num_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.Num_features = Num_features
        self.root = None

    def fit(self, X, y):
        #shape[1] = nuber of columns
        self.Num_features = X.shape[1] if not self.Num_features else min(self.Num_features, X.shape[1])
        self.root = self.grow(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def grow(self, X, y, depth=0):
        n_samples, n_features = X.shape
        
        #gives number of possible outcomes
        labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
                or labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        #pick random feature
        f_inds = np.random.choice(n_features, self.Num_features, replace=False)

        # select the best split using IG values
        best_feat, best_thresh = self._best_criteria(X, y, f_inds)
        
         # grow nodes from split
        li, ri = self._split(X[:, best_feat], best_thresh)
        left = self.grow(X[li, :], y[li], depth+1)
        right = self.grow(X[ri, :], y[ri], depth+1)
        return Node(best_feat, best_thresh, left, right)
    
#function to find the best threshold to use next for maximum information gain
    def _best_criteria(self, X, y, f_inds):
        best_gain = -1
        split_idx, split_thresh = None, None
        for f_ind in f_inds:
            X_column = X[:, f_ind]
            thresholds = np.unique(X_column)
            #[1,,2,3,4,5,6]
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = f_ind
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)
        # generate split
        li, ri = self._split(X_column, split_thresh)

        if len(li) == 0 or len(ri) == 0:
            return 0

        n = len(y)
        no_left, no_right = len(li), len(ri)
        ent_l, ent_r = entropy(y[li]), entropy(y[ri])
        child_entropy = (no_left / n) * ent_l + (no_right / n) * ent_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig
#function to split the indices to he ones les than and more tan the threshold
    def _split(self, X_column, split_thresh):
        li = np.argwhere(X_column <= split_thresh).flatten()
        ri = np.argwhere(X_column > split_thresh).flatten()
        return li, ri
#function to traverse down the tree
    def _traverse_tree(self, x, node):
        #we've reached the end of the tree, ths return a value
        if node.is_leaf():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
#used to find most common data type after split
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

In [17]:
def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

def pick_data(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]


#Class for Random Forest
class RandomForest:
    
    def __init__(self, n_trees=8, min_samples_split=1,
                 max_depth=100, Num_features=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.Num_features = Num_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split,
                max_depth=self.max_depth, Num_features=self.Num_features)
            X_samp, y_samp = pick_data(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        predictions = np.swapaxes(predictions, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in predictions]
        return np.array(y_pred)

In [18]:
import numpy as np

def accuracy(y_true, y_predicted):
    correct = np.sum(y_true == y_predicted) 
    accuracy = correct/ len(y_true)
    return accuracy

X_train = (convert(pd.read_csv("dataset_files/car_X_train.csv", header=0))).to_numpy()
X_test = (convert(pd.read_csv("dataset_files/car_X_test.csv", header=0))).to_numpy()

y_dict={ 'unacc':1, 'acc':2 ,'good':3, 'vgood':4}

y_train = pd.read_csv("dataset_files/car_y_train.csv", header=0)
y_train['class'] = y_train['class'].map(y_dict)
y_train = y_train.to_numpy()
y_train=y_train.flatten()

y_test = pd.read_csv("dataset_files/car_y_test.csv", header=0)
y_test['class'] = y_test['class'].map(y_dict)
y_test = y_test.to_numpy()
y_test=y_test.flatten()

clf = RandomForest(n_trees=7, max_depth=10)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)
df3 = acc.view()
df3

0.95761078998073212